In [11]:
import sqlalchemy
import pandas as pd

from transformers import RobertaTokenizer, RobertaModel

In [19]:
db_name = 'sex.db'
model_path = '../cmv/roberta_balanced/'
batch_size = 32

In [13]:
# create the connection
engine = sqlalchemy.create_engine('sqlite:///%s' % db_name)

# open engine connection
with engine.connect() as con:

    # perform query
    rs = con.execute("SELECT * FROM comments")

    # save results of the query to a dataframe
    df = pd.DataFrame(rs.fetchall())

# set the dataframe's column names to the corresponding names in the database
df.columns = rs.keys()


In [14]:
df.head(1)

,comment_id,submission_id,url,comment
0,j0slrg1,zpd34l,https://www.reddit.com/r/sex/comments/zpd34l/c...,wow you guys are amazing\n\nwhen I feel the ti...


In [17]:
# load the model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained(model_path)

Some weights of the model checkpoint at ../cmv/roberta_balanced/ were not used when initializing RobertaModel: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../cmv/roberta_balanced/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
for i in range(0, df.shape[0], batch_size):
    print(score_batch(df.text[i:i+batch_size]))

AttributeError: 'DataFrame' object has no attribute 'text'